<a href="https://colab.research.google.com/github/Pistolll/praktika2_TMOC/blob/main/1.1_prak_02_NB_SpamEmail_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Импорт основных библиотек для работы с данными
import numpy as np  # Для численных операций и работы с массивами
import pandas as pd  # Для работы с табличными данными (DataFrame)

# Импорт библиотек для визуализации
import matplotlib as mpl  # Базовый функционал matplotlib
import matplotlib.pyplot as plt  # Построение графиков
import seaborn as sns  # Улучшенные визуализации на основе matplotlib


# Импорт модулей для работы с текстом
import string  # Работа со строковыми операциями
from sklearn.feature_extraction.text import TfidfVectorizer  # Преобразование текста в TF-IDF признаки
from sklearn.feature_extraction.text import CountVectorizer  # Преобразование текста в мешок слов

# Импорт инструментов машинного обучения
from sklearn.model_selection import train_test_split  # Разделение данных на обучающую/тестовую выборки
from sklearn import preprocessing  # Предварительная обработка данных

# Импорт NLP-инструментов из NLTK
from nltk.stem import SnowballStemmer  # Стеммер для приведения слов к основе
from nltk.tokenize import RegexpTokenizer  # Токенизатор по регулярным выражениям
from nltk.corpus import stopwords  # Список стоп-слов
from nltk.stem.porter import *  # Поттер-стеммер
import nltk  # Основная NLP-библиотека
nltk.download('stopwords')  # Загрузка стоп-слов для NLTK
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
url = "https://raw.githubusercontent.com/Pistolll/praktika2_TMOC/refs/heads/main/%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0%202/01-custom-naive-bayes/spam.csv"
df = pd.read_csv(url, encoding='ISO-8859-1')
le = preprocessing.LabelEncoder() # категориальные метки в числовой формат

In [5]:
data = df.to_numpy() # DataFrame df в массив NumPy.

In [6]:
# Выделяем признаки (X) и целевую переменную (y) из данных:
X = data[:, 1]  # Берём все строки и только второй столбец (индекс 1) - это матрица признаков
y = data[:, 0]  # Берём все строки и первый столбец (индекс 0) - это вектор целевых значений

In [7]:
X.shape, y.shape

((5572,), (5572,))

In [8]:
# Инициализация токенизатора для разбиения текста на токены
# RegexpTokenizer('\w+') - разделяет текст только на слова (игнорирует пунктуацию)
# \w+ - регулярное выражение, означающее "1 или более буквенно-цифровых символов"
tokenizer = RegexpTokenizer('\w+')

# Стоп-слова - это частые, но малозначимые слова (артикли, предлоги и т.д.)
# set() преобразует список в множество для быстрого поиска
sw = set(stopwords.words('english'))

# Стеммер приводит слова к их корневой форме (например: "running" → "run")
ps = PorterStemmer()

In [10]:
def getStem(review):

    # Приводим весь текст к нижнему регистру для единообразия
    review = review.lower()

    # Разбиваем текст на отдельные слова (токены), игнорируя пунктуацию
    tokens = tokenizer.tokenize(review)

    # Удаляем стоп-слова (малозначимые слова, предлоги, артикли и т.д.)
    removed_stopwords = [w for w in tokens if w not in sw]

    # Применяем стемминг к каждому слову (приводим к корневой форме)
    stemmed_words = [ps.stem(token) for token in removed_stopwords]

    # Собираем обработанные слова обратно в единую строку
    clean_review = ' '.join(stemmed_words)

    return clean_review

In [11]:
# Функция для очистки и предварительной обработки набора текстовых документов. Возвращает список обработанных и очищенных документов
def getDoc(document):
    d = []  # Создаем пустой список для хранения обработанных документов

    # Проходим по каждому документу в переданной коллекции
    for doc in document:
        # Применяем функцию getStem для очистки и обработки каждого документа
        d.append(getStem(doc))

    return d  # Возвращаем список обработанных документов

In [12]:
# Получаем очищенные документы, применяя функцию getDoc к массиву X
stemmed_doc = getDoc(X)

In [13]:
# Выводим первые 10 очищенных и стеммированных документов
stemmed_doc[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send ã â 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea ã â 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030']

In [17]:
cv = CountVectorizer() # преобразует коллекцию текстовых документов в числовую матрицу, где строки представляют документы,
# а столбцы — уникальные слова (или токены) из этих документов. Каждый элемент матрицы указывает, сколько раз соответствующее слово встречается в документе.

In [18]:
# Создание словаря с помощью CountVectorizer
vc = cv.fit_transform(stemmed_doc)
#позволяет определить уникальные слова, которые будут использоваться для представления текстовых данных в числовом формате.

In [19]:
X = vc.todense() # преобразует матрицу, созданную с помощью CountVectorizer, в плотную матрицу

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) # разделяем на обучающую и тестовую выборки, 33% данных будут отведены для тестирования, а оставшиеся 67% — для обучения

In [21]:
# Импортирование класса MultinomialNB из библиотеки sklearn
from sklearn.naive_bayes import MultinomialNB
# модель наивного байесовского классификатора, что позволит классифицировать новые документы на основе обученных данных.

In [22]:
# Создание экземпляра модели наивного байесовского классификатора
model = MultinomialNB()

# Обучение модели на обучающем наборе данных
model.fit(np.asarray(X_train), y_train)

# Оценка точности модели на тестовом наборе данных
model.score(np.asarray(X_test), y_test)

0.977705274605764

In [23]:
messages = [
    """
    Hi Kunal,
We invite you to participate in MishMash - India’s largest online diversity hackathon.
The hackathon is a Skillenza initiative and sponsored by Microsoft, Unity, Unilever, Gojek, Rocketium and Jharkhand Government.
We have a special theme for you - Deep Tech/Machine Learning - sponsored by Unilever, which will be perfect for you.
    """,
    """Join us today at 12:00 PM ET / 16:00 UTC for a Red Hat DevNation tech talk on AWS Lambda and serverless Java with Bill Burke.
Have you ever tried Java on AWS Lambda but found that the cold-start latency and memory usage were far too high?
In this session, we will show how we optimized Java for serverless applications by leveraging GraalVM with Quarkus to
provide both supersonic startup speed and a subatomic memory footprint.""",

    """We really appreciate your interest and wanted to let you know that we have received your application.
There is strong competition for jobs at Intel, and we receive many applications. As a result, it may take some time to get back to you.
Whether or not this position ends up being a fit, we will keep your information per data retention policies,
so we can contact you for other positions that align to your experience and skill set.
"""
]

In [24]:
#преобразовать обработанные документы в матрицу частот слов.
def prepare(messages):
    d = getDoc(messages)
    # dont do fit_transform!! it will create new vocab.
    return cv.transform(d)
messages = prepare(messages)

In [25]:
# В этом коде используется метод predict модели model,
# чтобы сделать предсказания на основе подготовленных сообщений, которые были переданы в функцию prepare. Это позволяет модели классифицировать каждое сообщение в соответствии с обученными категориями
y_pred = model.predict(messages)
y_pred

array(['ham', 'spam', 'ham'], dtype='<U4')